In [15]:
import sys
sys.path.append('/Users/beyond/PycharmProjects/stockAnalysis')

from analysis_util.cal_stock_trend import cal_trend_common
from selected_stock_analysis.buy_point_detect import buy_point_detect
from trade_data.get_trade_data import get_stock_trade_data_latestdays
from trade_data.get_stock_basic_list import get_stock_basic_list
import pandas as pd
import numpy as np
from analysis_util.plot_k_line import plot_k_line, plot_k_line_df,plot_k_line_latestdays

In [2]:
# 效果追踪
# 获取买点csv文件中数据的日期，计算下一天的涨跌
df = pd.read_csv('上升通道回调.csv', dtype={'code': np.str_})
df.head()

,code,name,desc,date
0,600750,江中药业,接近均线5,20250107
1,600707,彩虹股份,接近均线10/20,20250107
2,603518,锦泓集团,接近均线5,20250107
3,000333,美的集团,接近均线5/10,20250107
4,000063,中兴通讯,接近均线20,20250107


In [22]:
df.count()

code    153
name    153
desc    153
date    153
dtype: int64

In [3]:
mm_l = []
for l in df.values[:]:
    code = l[0]
    name = l[1]
    desc = l[2]
    date = l[3]
    # 获取价格
    trade_df = get_stock_trade_data_latestdays(code, 100)
    trade_df = trade_df[trade_df.trade_date >= str(date)]
    # print(trade_df)
    k1 = cal_trend_common(trade_df['close'].values[0:2])
    # print('k1: ', k1)
    k2 = cal_trend_common(trade_df['close'].values[0:5])
    # print('k2: ', k2)

    if '5' in desc:
        mm_l.append([code, name, date,'m5',k1, k2])
    if '10' in desc:
        mm_l.append([code, name, date,'m10',k1, k2])
    if '20' in desc:
        mm_l.append([code, name, date,'m20',k1, k2])
    if '30' in desc:
        mm_l.append([code, name, date,'m30',k1, k2])

df_effect=pd.DataFrame(mm_l,columns=['code','name','date','mean','k1','k2'])
df_effect.head()

,code,name,date,mean,k1,k2
0,600750,江中药业,20250107,m5,-0.05,-0.299
1,600707,彩虹股份,20250107,m10,0.29,-0.111
2,600707,彩虹股份,20250107,m20,0.29,-0.111
3,603518,锦泓集团,20250107,m5,-0.24,-0.195
4,000333,美的集团,20250107,m5,1.65,-0.447


In [9]:
# 获取股票基础信息，做join
df_s_info=get_stock_basic_list()
df_s_info.head()

,ts_code,symbol,name,area,industry,cnspell,market,list_date,act_name,act_ent_type
0,000001.SZ,000001,平安银行,深圳,银行,payh,主板,19910403,无实际控制人,无
1,000002.SZ,000002,万科A,深圳,全国地产,wka,主板,19910129,深圳市人民政府国有资产监督管理委员会,地方国企
2,000004.SZ,000004,国华网安,深圳,软件服务,ghwa,主板,19910114,李映彤,民营企业
3,000006.SZ,000006,深振业A,深圳,区域地产,szya,主板,19920427,深圳市人民政府国有资产监督管理委员会,地方国企
4,000007.SZ,000007,全新好,深圳,其他商业,qxh,主板,19920413,王玩虹,民营企业


In [11]:
df2=pd.merge(df_effect,df_s_info,how='left')
df2.head()

,code,name,date,mean,k1,k2,ts_code,symbol,area,industry,cnspell,market,list_date,act_name,act_ent_type
0,600750,江中药业,20250107,m5,-0.05,-0.299,600750.SH,600750,江西,中成药,jzyy,主板,19960923,江西省国有资产监督管理委员会,地方国企
1,600707,彩虹股份,20250107,m10,0.29,-0.111,600707.SH,600707,陕西,元器件,chgf,主板,19960520,咸阳市国有资产监督管理委员会,地方国企
2,600707,彩虹股份,20250107,m20,0.29,-0.111,600707.SH,600707,陕西,元器件,chgf,主板,19960520,咸阳市国有资产监督管理委员会,地方国企
3,603518,锦泓集团,20250107,m5,-0.24,-0.195,603518.SH,603518,江苏,服饰,jhjt,主板,20141203,None,None
4,000333,美的集团,20250107,m5,1.65,-0.447,000333.SZ,000333,广东,家用电器,mdjt,主板,20130918,None,None


In [20]:
# m5
df_m30=df2[df2['mean']=='m30']

In [21]:
df_m30.groupby('industry').size()

industry
中成药     4
元器件     3
农药化肥    1
化工原料    1
化工机械    3
半导体     3
啤酒      2
家用电器    4
小金属     1
摩托车     2
服饰      2
汽车配件    7
通信设备    1
银行      6
食品      2
dtype: int64

In [17]:
df_m30=df2[df2['mean']=='m30'][df2['k2']>0]

In [18]:
df_m30.groupby('industry').size()

industry
中成药     2
元器件     1
化工机械    2
半导体     2
家用电器    2
汽车配件    5
银行      6
dtype: int64

In [16]:
df2[df2['mean']=='m30'][df2['k1']<0].count()

code            19
name            19
date            19
mean            19
k1              19
k2              19
ts_code         19
symbol          19
area            19
industry        19
cnspell         19
market          19
list_date       19
act_name        11
act_ent_type    11
dtype: int64

In [16]:
# plot_k_line_latestdays('000651',100)

In [12]:
df_effect[df_effect['mean']=='m30'][df_effect['k2']>0].count()

code    5
name    5
date    5
mean    5
k1      5
k2      5
dtype: int64

In [13]:
df = pd.read_csv('下降通道到关键均线1.csv', dtype={'code': np.str_})
mm_l = []
for l in df.values[:]:
    code = l[0]
    name = l[1]
    desc = l[2]
    date = l[3]
    # 获取价格
    trade_df = get_stock_trade_data_latestdays(code, 100)
    trade_df = trade_df[trade_df.trade_date >= str(date)]
    print(trade_df)
    k1 = cal_trend_common(trade_df['close'].values[0:2])
    print('k1: ', k1)
    k2 = cal_trend_common(trade_df['close'].values)
    print('k2: ', k2)

    if '60' in desc:
        mm_l.append([code, name, date,'m60',k1, k2])
    if '140' in desc:
        mm_l.append([code, name, date,'m140',k1, k2])
    if '250' in desc:
        mm_l.append([code, name, date,'m250',k1, k2])

df_effect=pd.DataFrame(mm_l,columns=['code','name','date','mean','k1','k2'])
df_effect.head()

      ts_code trade_date   open   high    low  close  pre_close  change  pct_chg        vol       amount  name
39  600529.SH   20241029  26.45  26.56  25.21  25.57      26.59   -1.02  -3.8360  371785.48   950230.016  山东药玻
40  600529.SH   20241030  25.45  25.72  25.15  25.43      25.57   -0.14  -0.5475  143239.00   363917.064  山东药玻
41  600529.SH   20241031  25.43  25.45  25.05  25.13      25.43   -0.30  -1.1797  151452.02   381023.471  山东药玻
42  600529.SH   20241101  25.12  25.34  24.85  25.00      25.13   -0.13  -0.5173  153091.56   383780.846  山东药玻
43  600529.SH   20241104  25.18  25.34  24.80  25.32      25.00    0.32   1.2800  167262.41   419252.159  山东药玻
44  600529.SH   20241105  25.20  25.62  25.02  25.53      25.32    0.21   0.8294  198683.66   504791.889  山东药玻
45  600529.SH   20241106  25.58  25.75  25.29  25.37      25.53   -0.16  -0.6267  179924.19   458612.276  山东药玻
46  600529.SH   20241107  25.25  25.81  25.10  25.81      25.37    0.44   1.7343  175469.36   449388.916  山东药玻
4

,code,name,date,mean,k1,k2
0,600529,山东药玻,20241029,m250,-0.140,-0.009640
1,601398,工商银行,20241029,m60,-0.049,0.015891
2,600750,江中药业,20241030,m60,-0.210,0.025175
3,600511,国药股份,20241030,m60,0.320,-0.030645
4,002304,洋河股份,20241030,m140,-4.240,-0.141713


In [58]:
# m60
df_effect[df_effect['mean']=='m140'].count()

code    162
name    162
date    162
mean    162
k1      162
k2      162
dtype: int64

In [59]:
df_effect[df_effect['mean']=='m140'][df_effect['k1']>0].count()

code    73
name    73
date    73
mean    73
k1      73
k2      73
dtype: int64

In [60]:
df_effect[df_effect['mean']=='m140'][df_effect['k2']>0].count()

code    88
name    88
date    88
mean    88
k1      88
k2      88
dtype: int64